In [1]:
import xarray as xr
import rioxarray
import plotly.express as px
import numpy as np

In [2]:
xds = xr.open_dataset("../../app/data/farsite.nc", decode_coords="all")
xds

<xarray.Dataset>
Dimensions:                    (x: 3397, y: 2910)
Coordinates:
    albers_conical_equal_area  |S1 ...
  * x                          (x) float64 -2.284e+06 -2.284e+06 ... -2.182e+06
  * y                          (y) float64 1.832e+06 1.832e+06 ... 1.919e+06
Data variables: (12/14)
    US_210CBD                  (y, x) float32 ...
    US_210CBH                  (y, x) float32 ...
    US_210CC                   (y, x) float32 ...
    US_210CH                   (y, x) float32 ...
    US_210EVC                  (y, x) float32 ...
    US_210EVH                  (y, x) float32 ...
    ...                         ...
    US_210FVH                  (y, x) float32 ...
    US_210FVT                  (y, x) float32 ...
    US_ASP                     (y, x) float32 ...
    US_DEM                     (y, x) float32 ...
    US_FDIST                   (y, x) float32 ...
    US_SLP                     (y, x) float32 ...
Attributes:
    GDAL_AREA_OR_POINT:  Area
    Conventions:         CF-1.5
    GDAL:                GDAL 3.3.2, released 2021/09/01
    history:             Fri Feb  4 09:00:45 2022: ncrename -v Band14,US_SLP ...
    NCO:                 netCDF Operators version 4.9.1 (Homepage = http://nc...

In [3]:
# epsg 3857 bounds
xds_3857 = xds.rio.reproject("EPSG:3857")
xds_3857.x.min().item(), xds_3857.y.min().item(), xds_3857.x.max().item(), xds_3857.y.max().item()

(-13625507.8062414, 4396202.443458099, -13471703.74783939, 4534975.565295047)

In [4]:
xds_lonlat = xds.rio.reproject("EPSG:4326")
xds_lonlat

<xarray.Dataset>
Dimensions:                    (x: 4724, y: 3396)
Coordinates:
  * x                          (x) float64 -122.4 -122.4 ... -121.0 -121.0
  * y                          (y) float64 37.68 37.68 37.68 ... 36.69 36.69
    albers_conical_equal_area  int64 0
Data variables: (12/14)
    US_210CBD                  (y, x) float32 nan nan nan nan ... nan nan nan
    US_210CBH                  (y, x) float32 nan nan nan nan ... nan nan nan
    US_210CC                   (y, x) float32 nan nan nan nan ... nan nan nan
    US_210CH                   (y, x) float32 nan nan nan nan ... nan nan nan
    US_210EVC                  (y, x) float32 nan nan nan nan ... nan nan nan
    US_210EVH                  (y, x) float32 nan nan nan nan ... nan nan nan
    ...                         ...
    US_210FVH                  (y, x) float32 nan nan nan nan ... nan nan nan
    US_210FVT                  (y, x) float32 nan nan nan nan ... nan nan nan
    US_ASP                     (y, x) float32 nan nan nan nan ... nan nan nan
    US_DEM                     (y, x) float32 nan nan nan nan ... nan nan nan
    US_FDIST                   (y, x) float32 nan nan nan nan ... nan nan nan
    US_SLP                     (y, x) float32 nan nan nan nan ... nan nan nan
Attributes:
    GDAL_AREA_OR_POINT:  Area
    Conventions:         CF-1.5
    GDAL:                GDAL 3.3.2, released 2021/09/01
    history:             Fri Feb  4 09:00:45 2022: ncrename -v Band14,US_SLP ...
    NCO:                 netCDF Operators version 4.9.1 (Homepage = http://nc...

In [5]:
# epsg 4326 bounds
xds_lonlat.x.min().item(), xds_lonlat.y.min().item(), xds_lonlat.x.max().item(), xds_lonlat.y.max().item()

(-122.4000416897913, 36.69159239155589, -121.01852570346, 37.68465755640763)

In [6]:
xds_lonlat_low = xds_lonlat.coarsen(dim={"x": 16, "y": 16}, boundary="trim").mean()
xds_lonlat_low

<xarray.Dataset>
Dimensions:                    (y: 212, x: 295)
Coordinates:
  * x                          (x) float64 -122.4 -122.4 ... -121.0 -121.0
  * y                          (y) float64 37.68 37.68 37.67 ... 36.7 36.7 36.69
    albers_conical_equal_area  int64 0
Data variables: (12/14)
    US_210CBD                  (y, x) float32 nan nan nan nan ... nan nan nan
    US_210CBH                  (y, x) float32 nan nan nan nan ... nan nan nan
    US_210CC                   (y, x) float32 nan nan nan nan ... nan nan nan
    US_210CH                   (y, x) float32 nan nan nan nan ... nan nan nan
    US_210EVC                  (y, x) float32 nan nan nan nan ... nan nan nan
    US_210EVH                  (y, x) float32 nan nan nan nan ... nan nan nan
    ...                         ...
    US_210FVH                  (y, x) float32 nan nan nan nan ... nan nan nan
    US_210FVT                  (y, x) float32 nan nan nan nan ... nan nan nan
    US_ASP                     (y, x) float32 nan nan nan nan ... nan nan nan
    US_DEM                     (y, x) float32 nan nan nan nan ... nan nan nan
    US_FDIST                   (y, x) float32 nan nan nan nan ... nan nan nan
    US_SLP                     (y, x) float32 nan nan nan nan ... nan nan nan
Attributes:
    GDAL_AREA_OR_POINT:  Area
    Conventions:         CF-1.5
    GDAL:                GDAL 3.3.2, released 2021/09/01
    history:             Fri Feb  4 09:00:45 2022: ncrename -v Band14,US_SLP ...
    NCO:                 netCDF Operators version 4.9.1 (Homepage = http://nc...

In [7]:
df = xds_lonlat_low.to_dataframe()
df.reset_index(inplace=True)
df.dropna(inplace=True)
df.head()

,y,x,US_210CBD,US_210CBH,US_210CC,US_210CH,US_210EVC,US_210EVH,US_210F40,US_210FVC,US_210FVH,US_210FVT,US_ASP,US_DEM,US_FDIST,US_SLP,albers_conical_equal_area
234,37.682464,-121.302697,0.000000,0.000000,0.000000,0.000000,68.000000,68.000000,93.000000,68.000000,68.000000,2968.00000,-1.0,17.000000,0.0,0.000000,0
235,37.682464,-121.298017,0.000000,0.000000,0.000000,0.000000,121.878052,117.414635,98.756096,79.609756,148.682922,2834.14624,-1.0,17.146341,0.0,0.292683,0
236,37.682464,-121.293337,0.718750,31.093750,10.781250,47.395832,98.739586,95.291664,99.468750,91.718750,455.114594,2962.25000,-1.0,16.375000,0.0,0.166667,0
237,37.682464,-121.288657,1.000000,34.660000,15.000000,53.200001,110.313332,104.806664,102.000000,101.000000,605.320007,2960.00000,-1.0,14.886666,0.0,0.246667,0
238,37.682464,-121.283977,0.289216,6.818627,4.338235,10.637255,124.578430,114.387253,99.524513,69.450981,288.779419,2932.14209,-1.0,13.377451,0.0,0.387255,0


In [8]:
df.describe()

,y,x,US_210CBD,US_210CBH,US_210CC,US_210CH,US_210EVC,US_210EVH,US_210F40,US_210FVC,US_210FVH,US_210FVT,US_ASP,US_DEM,US_FDIST,US_SLP,albers_conical_equal_area
count,41693.000000,41693.000000,41693.000000,41693.000000,41693.000000,41693.000000,41693.000000,41693.000000,41693.000000,41693.000000,41693.000000,41693.000000,41693.000000,41693.000000,41693.000000,41693.000000,41693.0
mean,37.189227,-121.706820,2.673118,18.198957,17.415329,91.671585,162.875015,143.222427,124.192200,83.208458,377.890350,2075.623535,128.991013,310.380829,4.927210,12.380216,0.0
std,0.229242,0.332982,4.664359,23.040964,21.273232,99.059875,99.457428,88.633308,36.145435,39.813202,224.415421,808.010315,88.018288,272.697021,30.824942,9.020273,0.0
min,36.694956,-122.397848,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,-0.031250,0.000000,0.000000,0.0
25%,36.999165,-121.985996,0.011719,0.062500,0.136719,0.742188,72.476562,67.207031,101.988281,51.343750,131.335938,2108.570312,29.566406,62.859375,0.000000,1.429688,0.0
50%,37.191050,-121.705189,0.773438,6.015625,8.144531,52.187500,171.125000,144.234375,119.496094,105.347656,466.363281,2193.101562,146.371094,254.718750,0.000000,14.296875,0.0
75%,37.378255,-121.424381,2.500000,32.785156,27.675781,170.000000,231.781250,207.937500,147.757812,111.453125,557.753906,2402.734375,200.910156,497.285156,0.000000,19.574219,0.0
max,37.682464,-121.021890,24.210938,100.000000,78.515625,450.468750,354.992188,306.203125,189.000000,125.027344,644.968750,2968.000000,318.000000,1239.101562,612.000000,35.964844,0.0


In [9]:
xds_lonlat.to_netcdf("../../app/data/farsite_lonlat.nc")
xds_lonlat_low.to_netcdf("../../app/data/farsite_lonlat_low.nc")
df.to_csv("../../app/data/farsite_lonlat_low.csv", index=False)